In [ ]:
import transformers, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Part1: run the demo

- author: lyuchuny3@foxmail.com
- 知乎-link: https://zhuanlan.zhihu.com/p/1920540188146373944

## llama2-7b-half
关于demo代码的几个说明：
1. device_map="auto", 因为我用的是CPU，没有用CUDA
2. 数据类型：用torch.bfloat16
3. 生成速度太慢：我修改了 max_new_tokens=48，这个值越小，跑得越快，答案也越简短
4. 模型路径："./Chinese-LLaMA-2-7B-hf", 我下载到执行的当前文件夹
5. 需要python环境安装了transformers, torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./Chinese-LLaMA-2-7B-hf", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "./Chinese-LLaMA-2-7B-hf",  #########
    device_map="auto",          #########
    torch_dtype=torch.bfloat16, #########
    trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [01:31<00:00, 45.85s/it]


In [3]:
prompt = '我喜欢攀岩'
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, 
                              do_sample=True, 
                              max_new_tokens=48, ########
                              top_k=10, 
                              top_p=0.85, 
                              temperature=1, 
                              repetition_penalty=1.15, 
                              eos_token_id=2, 
                              bos_token_id=1, 
                              pad_token_id=0)
response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
response = response.lstrip(prompt)
print(response)

/home/chun/.conda/envs/py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


，想要一个专业的教练。


## 大模型推理Inference流程分析


### 1. Tokenizer：对输入句子进行分词
分词阶段，分词器tokenizer对raw_text_input进行分词，变成一个一个的token， 然后映射到整数id（在词表vocab_size的范围内）。再把input_id给大模型作为输入。（分词阶段一般不包含embedding, embedding一般是在模型里做的）

在分词阶段：

- encode: raw_text->input_id 把输入文本变成token_id。会在开头添加 bos(句子开始标记），在末尾添加eos句子结束标记
- decode: output_id ->raw_string 把输出的token_id重新映射为文本

Note:
- bos, eos它们是 NLP 中常用的​​特殊 token（特殊符号）​​，用于标识一个句子的起始和结束，在很多预训练语言模型中被广泛使用。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./Chinese-LLaMA-2-7B-hf", use_fast=False, trust_remote_code=True)
type(tokenizer)

transformers.models.llama.tokenization_llama.LlamaTokenizer

In [ ]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
inputs.input_ids

tensor([[    1, 29871, 30672, 31823, 33205, 33646, 31753]])

In [ ]:
raw_string = tokenizer.batch_decode(inputs.input_ids, skip_special_tokens=True)[0]
print(raw_string)

我喜欢攀岩


In [ ]:
raw_string = tokenizer.batch_decode(inputs.input_ids)[0]
print(raw_string)

<s> 我喜欢攀岩


### 2. 加载大模型
可以看到，这里加载的模型是 CausalLM（Causal Language Model）, 因果语言模型。

transformer模型主要有3种架构，不同任务可以采用不同架构：

- encoder-decoder: 翻译任务，seq2seq
- encoder-only: 掩码模型，自编码模型（auto-encoding models), 预测句子种被mask掉的token
- decoder-only: 因果语言模型，自回归模型，文本生成（GPT, Llama), 从左到右预测下一tokens
目前LLaMA, GPT都属于decoder-only: 自回归模型（auto-regressive models）适合做生成任务，也被称为因果语言模型（Causal Language Model) ，因为这类模型从左到右预测下一个token, 从已知的一系列tokens获取context，预测下一个token。该建模方法遵循因果原则，当前单词只受到之前单词的影响，而不受后面单词的影响。

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "./Chinese-LLaMA-2-7B-hf", 
    device_map="auto", 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True)   
type(model)


Loading checkpoint shards: 100%|██████████| 2/2 [01:28<00:00, 44.01s/it]


transformers.models.llama.modeling_llama.LlamaForCausalLM

### 3. 调用 model.generate() 开始生成

In [ ]:
generate_ids = model.generate(
    inputs.input_ids, 
    do_sample=True, 
    max_new_tokens=48, 
    top_k=10, 
    top_p=0.85, 
    temperature=1, 
    repetition_penalty=1.15, 
    eos_token_id=2, 
    bos_token_id=1, 
    pad_token_id=0)

In [ ]:
generate_ids

tensor([[    1, 29871, 30672, 31823, 33205, 33646, 31753, 30330, 32138, 33299,
         30330, 32552, 35043, 33373, 30214, 31994, 30417, 30505, 30716, 30429,
         32020, 32114, 33868, 31184, 31704, 30846, 30267, 30847, 30801, 30919,
         30392, 30287, 30502, 32274, 32911, 30613, 30214, 31356, 31882, 30919,
         30682, 30815, 30953, 32014, 32288, 33826, 30810, 31959, 31888, 30895,
         30267,    13, 29902,  5360, 10784]])

### 4. 得到输出句子
最后就是把生成的generate_ids 映射为字符，得到生成的句子

- encode: raw_text->input_id 把输入文本变成token_id
- decode: output_id ->raw_string 把输出的token_id重新映射为文本

In [ ]:
response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
response = response.lstrip(prompt)
print(response)

、冲浪、滑翔伞，还有在水上做飞碟等活动。如果你是一个冒险家，那么你可能也很熟悉这些项目。
I love clim
